In [1]:
import cv2
import numpy as np
import glob  # for read file from folder
import time  # for estimate processing time(optional)
import matplotlib.pyplot as plt

In [2]:
def plot_gray(image):
    plt.figure(figsize=(20,15))
    return plt.imshow(image, cmap='Greys_r')

In [6]:
def find_features(img1):
    correct_matches_dct = {}
    directory = 'images/cards/sample/'
    for image in os.listdir(directory):
        img2 = cv2.imread(directory+image, 0)
        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(img1, None)
        kp2, des2 = orb.detectAndCompute(img2, None)
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
        correct_matches = []
        for m, n in matches:
            if m.distance < 0.75*n.distance:
                correct_matches.append([m])
                correct_matches_dct[image.split('.')[0]]= len(correct_matches)
    correct_matches_dct = dict(sorted(correct_matches_dct.items(),
                                     key=lambda item: item[1],
                                     reverse=True))
    return list(correct_matches_dct.keys())[0]